# 06. MLOps: Performance Monitoring

This notebook calculates model performance metrics from logged predictions.

## Setup

In [ ]:
import hopsworks
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Connect to Hopsworks
project = hopsworks.login()
fs = project.get_feature_store()

## Configuration

In [ ]:
# Configuration
feature_view_name = 'demand_features_view'
version = 1

## Read Prediction Logs

In [ ]:
# Get feature view
demand_fv = fs.get_feature_view(name=feature_view_name, version=version)

# Read prediction logs
prediction_logs = demand_fv.read_log()
print(f"Found {len(prediction_logs)} logged predictions")
prediction_logs.head()

## Calculate Performance Metrics

In [ ]:
# In production, you would join with actual values
# For demo, simulate actuals as predictions with some noise
prediction_logs['actual_demand'] = prediction_logs['prediction'] * np.random.normal(1.0, 0.1, len(prediction_logs))

# Calculate RMSE and MAE
rmse = np.sqrt(mean_squared_error(prediction_logs['actual_demand'], prediction_logs['prediction']))
mae = mean_absolute_error(prediction_logs['actual_demand'], prediction_logs['prediction'])

print(f"Overall Performance:")
print(f"  RMSE: {rmse:.2f}")
print(f"  MAE:  {mae:.2f}")

## Store Performance Metrics

In [ ]:
# Create a simple performance metrics dataframe
import datetime

performance_metrics = pd.DataFrame([{
    'timestamp': datetime.datetime.now(),
    'rmse': rmse,
    'mae': mae,
    'n_predictions': len(prediction_logs)
}])

# Create/get performance metrics feature group
performance_fg = fs.get_or_create_feature_group(
    name="model_performance_metrics",
    version=1,
    description="Model performance metrics over time",
    primary_key=['timestamp']
)

# Insert metrics
performance_fg.insert(performance_metrics)
print("✓ Stored performance metrics")

## Summary

✅ Calculated and stored model performance metrics

What we did:
- Read logged predictions from feature view
- Calculated RMSE and MAE (with simulated actuals)
- Stored metrics in a feature group for tracking

In production:
- Join predictions with actual values when available
- Run this as a scheduled job
- Monitor performance trends over time